In [1]:
import os
import json
from dotenv import load_dotenv  
import re
from cerebras.cloud.sdk import Cerebras
from prompts import system_prompt, user_prompt
load_dotenv()

# Initialize Cerebras client
client = Cerebras(
    api_key=os.environ.get("CEREBRAS_API_KEY"),
)

In [2]:
def close_door(door_id):
    return "Door closed for id: " + door_id

def open_door(door_id):
    return "Door opened for id: " + door_id


tools = [
    {
        "type": "function",
        "function": {
            "name": "close_door",
            "strict": True,
            "description": "A tool that can close a door. Use this when you need to close a door.",
            "parameters": {
                "type": "object",
                "properties": {
                    "door_id": {
                        "type": "string",
                        "description": "The id of the door to close"
                    }
                },
                "required": ["door_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "open_door",
            "strict": True,
            "description": "A tool that can open a door. Use this when you need to open a door.",
            "parameters": {
                "type": "object",
                "properties": {
                    "door_id": {
                        "type": "string",
                        "description": "The id of the door to open" 
                    }
                },
                "required": ["door_id"]
            }
        }
    }
]


In [3]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt.format(user_speech="Help there is someone chasing me.")}]

response = client.chat.completions.create(
    model="llama-4-scout-17b-16e-instruct",
    messages=messages,
    tools=tools,
    parallel_tool_calls=False,
)

In [4]:
choice = response.choices[0].message

if choice.tool_calls:
    function_call = choice.tool_calls[0].function
    if function_call.name == "open_door":
        # Logging that the model is executing a function to open a door
        print(f"Model executing function '{function_call.name}' with arguments {function_call.arguments}")

        # Parse the arguments from JSON format and perform the door opening
        arguments = json.loads(function_call.arguments)
        door_id = arguments["door_id"]
        print(f"Opening door with ID: {door_id}")
        
        # Send the result back to the model to fulfill the request
        messages.append({
            "role": "tool",
            "content": f"Door {door_id} has been opened successfully",
            "tool_call_id": choice.tool_calls[0].id
        })
    elif function_call.name == "close_door":
        # Logging that the model is executing a function to close a door
        print(f"Model executing function '{function_call.name}' with arguments {function_call.arguments}")

        # Parse the arguments from JSON format and perform the door closing
        arguments = json.loads(function_call.arguments)
        door_id = arguments["door_id"]
        print(f"Closing door with ID: {door_id}")
        
        # Send the result back to the model to fulfill the request
        messages.append({
            "role": "tool",
            "content": f"Door {door_id} has been closed successfully",
            "tool_call_id": choice.tool_calls[0].id
        })
 
    # Request the final response from the model, now that it has the tool result
    final_response = client.chat.completions.create(
        model="llama-4-scout-17b-16e-instruct",
        messages=messages,
    )
    
    # Handle and display the model's final response
    if final_response:
        print("Final model output:", final_response.choices[0].message.content)
    else:
        print("No final response received")
else:
    # Handle cases where the model's response does not include expected tool calls
    print("Model response without tool calls:", choice.content)

Model executing function 'open_door' with arguments {"door_id": "nearest exit"}
Opening door with ID: nearest exit
Final model output: I'm here to help you, and you're doing great. Can you tell me where you are right now in the building? What's your current location?
